Здесь будут приведены самые базовые SQL-операции, использующие в повседневной практике.
Будет использован SQLLite диалект.

# Создание базы данных

Представим, что нам нужно спроектировать базу данных интернет-магазина.

In [29]:
# Создадим базу данных, которая будет храниться в оперативной памяти
from sql import create_connection_sqlite
conn = create_connection_sqlite()

2.6.0


# Создание связанных таблиц с внешними ключами

In [30]:
# Теперь будем проектировать таблицы
cur = conn.cursor()
# Мы хотим выполнить больше одного действия за раз, поэтому юзаем executescript
cur.executescript(
"""
CREATE TABLE customer
    (
    id integer primary key autoincrement,
    name varchar(255),
    phone varchar(30),
    email varchar(255)
    );

CREATE TABLE product
    (
    id integer primary key autoincrement,
    name varchar(255),
    description text,
    price integer
    );

CREATE TABLE product_photo
    (
    id integer primary key autoincrement,
    url varchar(255),
    product_id integer references product(id)

    );

CREATE TABLE cart
    (
    customer_id integer references customer(id),
    id integer primary key autoincrement
    );

CREATE TABLE cart_product
    (
    cart_id integer references cart(id),
    product_id integer references product(id)
    );
"""
)

In [31]:
# Посмотрим на схему нашей базы данных
cur = conn.cursor()
cur.execute("""
SELECT
    name
FROM
    sqlite_schema
""")
cur.fetchall()

[('customer',),
 ('sqlite_sequence',),
 ('product',),
 ('product_photo',),
 ('cart',),
 ('cart_product',)]

Описание таблиц:

- cart_product - связь между продуктом и заказом
- cart - заказ в интернет-магазине
- product_photo - фотография
- product - информация о товаре
- customer - информация о клиенте

# Вставим значения в таблицы

In [32]:
cur = conn.cursor()
cur.executescript(
"""
INSERT INTO customer(name, phone, email) values ('Василий', '02', 'vas@gmail.com');
INSERT INTO customer(name, phone, email) values ('Петр', '03', 'petr@gmail.com')
""")

In [33]:
# Посмотрим на наши значения
conn.cursor().execute("""SELECT * FROM customer;""").fetchall()

[(1, 'Василий', '02', 'vas@gmail.com'), (2, 'Петр', '03', 'petr@gmail.com')]

In [34]:
# Подгрузим pandas
import pandas as pd
pd.read_sql("""SELECT * FROM customer;""", conn)

,id,name,phone,email
0,1,Василий,02,vas@gmail.com
1,2,Петр,03,petr@gmail.com


In [35]:
cur = conn.cursor()
cur.executescript(
"""
INSERT INTO product(name, description, price) values ('iPhone', 'Cool phone', 100000);
INSERT INTO product(name, description, price) values ('Apple Watch', 'Cool watch', 50000)
""")

In [36]:
cur = conn.cursor()
cur.executescript(
"""
INSERT INTO product_photo(url, product_id) values ('iphone photo', 1);
""")

In [37]:
cur = conn.cursor()
cur.executescript(
"""
INSERT INTO cart(customer_id) values (1);
INSERT INTO cart_product(cart_id, product_id) values (1, 1), (1, 2);
""")

# Выбрать строки из таблицы по условию и соединить с другой таблицей

In [39]:
query = """
SELECT p.id, url, product_id, name, description, price
FROM product_photo pp
LEFT JOIN product p
ON p.id = pp.id
"""
print(pd.read_sql_query(query, conn))

   id           url  product_id    name description   price
0   1  iphone photo           1  iPhone  Cool phone  100000


![](https://i.ytimg.com/vi/GblsYkH4GLc/maxresdefault.jpg)

# Изменить запись в таблице

In [40]:
cur = conn.cursor()
cur.executescript(
"""
UPDATE product_photo set url = 'iphone_image_2' where id = 1
""")

# Множественное соединение

In [41]:
q = """
SELECT c.name, cart.id as cart_id, cp.product_id, p.price
FROM customer c
LEFT JOIN cart ON cart.customer_id = c.id
LEFT JOIN cart_product cp ON cp.cart_id = cart.id
LEFT JOIN product p ON p.id = cp.product_id
"""
pd.read_sql_query(q, conn)

,name,cart_id,product_id,price
0,Василий,1.0,1.0,100000.0
1,Василий,1.0,2.0,50000.0
2,Петр,NaN,NaN,NaN


# Добавим агрегацию и сортировку

In [44]:
q = """
SELECT c.name, coalesce(sum(p.price), 0) as orders_sum
FROM customer c
LEFT JOIN cart ON cart.customer_id = c.id
LEFT JOIN cart_product cp ON cp.cart_id = cart.id
LEFT JOIN product p ON p.id = cp.product_id
GROUP BY c.name
ORDER BY orders_sum
DESC
"""
pd.read_sql_query(q, conn)

,name,orders_sum
0,Василий,150000
1,Петр,0


# Обычная фильтрация

In [47]:
query = \
"""
SELECT *
FROM customer
WHERE LENGTH(name) > 4
"""
pd.read_sql_query(query, conn)

,id,name,phone,email
0,1,Василий,02,vas@gmail.com


# Добавим фильтрацию к агрегации

In [48]:
q = """
SELECT c.name, coalesce(sum(p.price), 0) as orders_sum
FROM customer c
LEFT JOIN cart ON cart.customer_id = c.id
LEFT JOIN cart_product cp ON cp.cart_id = cart.id
LEFT JOIN product p ON p.id = cp.product_id
GROUP BY c.name
HAVING sum(p.price) > 0
"""
pd.read_sql_query(q, conn)

,name,orders_sum
0,Василий,150000


Основное отличие `HAVING` от `WHERE` заключается в том, что в первом случае мы фильтруем группы.
Используется в агрегациях!